In [4]:
import random

class HQCScheme:
    """
    Реализация криптосистемы HQC (вариант с кодом повторения).
    Адаптирована для лабораторной работы.
    """
    def __init__(self, length=127, w_key=5, w_rand=5, w_err=5):
        self.N = length
        self.W_k = w_key
        self.W_r = w_rand
        self.W_e = w_err

        # Порог для декодера (majority vote)
        self.threshold = self.N // 2

        # Расчет теоретического максимума шума для проверки
        # Шум = f*v + u*g + e. Макс вес <= w*wr + wr*w + we
        max_noise = (self.W_k * self.W_r) + (self.W_r * self.W_k) + self.W_e
        is_safe = max_noise <= self.threshold

        print(f"=== Инициализация параметров HQC ===")
        print(f"  Длина кода (N): {self.N}")
        print(f"  Веса параметров: w={self.W_k}, wr={self.W_r}, we={self.W_e}")
        print(f"  Корректирующая способность (delta): {self.threshold}")
        print(f"  Теоретический макс. шум: {max_noise}")
        print(f"  Условие корректности ({max_noise} <= {self.threshold}): {is_safe}")
        print("="*60)

    # --- Утилиты ---

    def _weight(self, vec):
        """Возвращает вес Хэмминга."""
        return sum(vec)

    def _xor(self, a, b):
        """Побитовое сложение (XOR)."""
        return [x ^ y for x, y in zip(a, b)]

    def _convolve(self, a, b):
        """Циклическая свертка (умножение полиномов)."""
        res = [0] * self.N
        ones_a = [i for i, val in enumerate(a) if val]
        ones_b = [i for i, val in enumerate(b) if val]

        for i in ones_a:
            for j in ones_b:
                k = (i + j) % self.N
                res[k] ^= 1
        return res

    def _gen_poly(self, weight=None):
        """Генерация полинома: разреженного (если задан вес) или плотного."""
        vec = [0] * self.N
        if weight:
            for idx in random.sample(range(self.N), weight):
                vec[idx] = 1
        else:
            vec = [random.randint(0, 1) for _ in range(self.N)]
        return vec

    # --- Основные операции ---

    def keygen(self, verbose=False):
        """Генерация открытого и закрытого ключей."""
        if verbose:
            print("\n[1] Генерация ключей...")

        f = self._gen_poly(self.W_k)
        g = self._gen_poly(self.W_k)
        h = self._gen_poly() # Плотный полином

        # s = f + g*h
        s = self._xor(f, self._convolve(g, h))

        if verbose:
            print(f"  -> Секретный f (вес): {self._weight(f)} (цель: {self.W_k})")
            print(f"  -> Секретный g (вес): {self._weight(g)} (цель: {self.W_k})")
            print(f"  -> Публичный h (вес): {self._weight(h)} (случайный)")
            print(f"  -> Публичный s (вес): {self._weight(s)}")

        return (h, s), (f, g)

    def encrypt(self, bit, pk, verbose=False):
        """Шифрование бита m."""
        h, s = pk

        if verbose:
            print(f"\n[2] Шифрование бита: {bit}")

        # Генерация векторов эфемерных ключей и ошибки
        u = self._gen_poly(self.W_r)
        v = self._gen_poly(self.W_r) # v соответствует w из методички
        e = self._gen_poly(self.W_e)

        # c1 = u + h*v
        c1 = self._xor(u, self._convolve(h, v))

        # c2 = Code(m) + s*v + e
        m_vec = [bit] * self.N # Код повторения
        c2 = self._xor(self._xor(m_vec, self._convolve(s, v)), e)

        if verbose:
            print(f"  -> Рандомизатор u (вес): {self._weight(u)} (цель: {self.W_r})")
            print(f"  -> Рандомизатор v (вес): {self._weight(v)} (цель: {self.W_r})")
            print(f"  -> Вектор ошибки e (вес): {self._weight(e)} (цель: {self.W_e})")
            print(f"  -> Шифртекст c1 (вес): {self._weight(c1)}")
            print(f"  -> Шифртекст c2 (вес): {self._weight(c2)}")

        return c1, c2

    def decrypt(self, c_text, sk, verbose=False):
        """Расшифрование."""
        c1, c2 = c_text
        f, g = sk

        if verbose:
            print("\n[3] Расшифрование...")

        # Снятие маски: c_prime = c2 + c1*g
        # Это должно дать Code(m) + Шум
        c_prime = self._xor(c2, self._convolve(c1, g))

        # Анализ полученного вектора
        ones_count = self._weight(c_prime)
        res_bit = 1 if ones_count > self.threshold else 0

        if verbose:
            print(f"  -> Вес вектора после снятия маски (c'): {ones_count}")
            print(f"  -> Порог принятия решения (N/2): {self.threshold}")
            print(f"  -> Результат декодера: {res_bit}")

        return res_bit

    # --- Сценарии работы ---

    def demonstrate_full_cycle(self):
        """Демонстрация одного полного цикла с подробным выводом."""
        print("ЗАПУСК: Демонстрация работы алгоритма")
        print("-" * 40)

        # Генерация (verbose=True включает подробный вывод)
        pk, sk = self.keygen(verbose=True)

        # Шифрование
        target_msg = 1
        ciphertext = self.encrypt(target_msg, pk, verbose=True)

        # Расшифрование
        decrypted = self.decrypt(ciphertext, sk, verbose=True)

        print("\n[4] Итог операции")
        print(f"  Отправлено: {target_msg}")
        print(f"  Получено:   {decrypted}")
        print(f"  Статус:     {'УСПЕШНО' if target_msg == decrypted else 'ОШИБКА'}")

    def analyze_noise_structure(self):
        """Детальный анализ составляющих шума."""
        print("\n" + "="*60)
        print("АНАЛИЗ: Проверка структуры шума")
        print("="*60)

        # Создаем компоненты вручную для проверки формулы
        f = self._gen_poly(self.W_k)
        g = self._gen_poly(self.W_k)
        u = self._gen_poly(self.W_r)
        v = self._gen_poly(self.W_r)
        e = self._gen_poly(self.W_e)

        # Шум = f*v + u*g + e
        fv = self._convolve(f, v)
        ug = self._convolve(u, g)
        noise_part = self._xor(fv, ug)
        total_noise = self._xor(noise_part, e)

        w_total = self._weight(total_noise)

        print(f"Разложение шума (noise = f*v + u*g + e):")
        print(f"  1. Вес (f * v): {self._weight(fv)} (макс {self.W_k * self.W_r})")
        print(f"  2. Вес (u * g): {self._weight(ug)} (макс {self.W_r * self.W_k})")
        print(f"  3. Вес ошибки e: {self._weight(e)}")
        print(f"----------------------------------------")
        print(f"  Итоговый вес шума: {w_total}")
        print(f"  Допустимый предел: {self.threshold}")
        print(f"  Результат проверки: {'КОРРЕКТНО' if w_total <= self.threshold else 'СБОЙ'}")

    def run_tests(self, count=10):
        """Запуск серии тестов без лишнего вывода."""
        print("\n" + "="*60)
        print(f"ТЕСТИРОВАНИЕ: Серия из {count} проверок")
        print("="*60)

        success = 0
        for i in range(count):
            pk, sk = self.keygen(verbose=False)
            msg = random.randint(0, 1)
            c_text = self.encrypt(msg, pk, verbose=False)
            res = self.decrypt(c_text, sk, verbose=False)

            status = "OK" if msg == res else "FAIL"
            if msg == res: success += 1
            print(f"Тест {i+1:02d}: {msg} -> {res} | {status}")

        print("-" * 40)
        print(f"Успешно пройдено: {success}/{count} ({success/count*100}%)")

def main():
    # Создание экземпляра
    hqc = HQCScheme()

    # 1. Подробная демонстрация
    hqc.demonstrate_full_cycle()

    # 2. Анализ шума
    hqc.analyze_noise_structure()

    # 3. Тесты
    hqc.run_tests(10)

if __name__ == "__main__":
    main()

=== Инициализация параметров HQC ===
  Длина кода (N): 127
  Веса параметров: w=5, wr=5, we=5
  Корректирующая способность (delta): 63
  Теоретический макс. шум: 55
  Условие корректности (55 <= 63): True
ЗАПУСК: Демонстрация работы алгоритма
----------------------------------------

[1] Генерация ключей...
  -> Секретный f (вес): 5 (цель: 5)
  -> Секретный g (вес): 5 (цель: 5)
  -> Публичный h (вес): 82 (случайный)
  -> Публичный s (вес): 65

[2] Шифрование бита: 1
  -> Рандомизатор u (вес): 5 (цель: 5)
  -> Рандомизатор v (вес): 5 (цель: 5)
  -> Вектор ошибки e (вес): 5 (цель: 5)
  -> Шифртекст c1 (вес): 69
  -> Шифртекст c2 (вес): 67

[3] Расшифрование...
  -> Вес вектора после снятия маски (c'): 90
  -> Порог принятия решения (N/2): 63
  -> Результат декодера: 1

[4] Итог операции
  Отправлено: 1
  Получено:   1
  Статус:     УСПЕШНО

АНАЛИЗ: Проверка структуры шума
Разложение шума (noise = f*v + u*g + e):
  1. Вес (f * v): 23 (макс 25)
  2. Вес (u * g): 21 (макс 25)
  3. Вес ошибк